In [1]:
%matplotlib notebook
import os
import os.path as path
import numpy as np
from scipy.optimize import curve_fit
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import h5py

from pyccapt.calibration.calibration_tools import tools, data_tools, variables, tof_calibration, data_loadcrop
from pyccapt.calibration.calibration_tools import widgets as wd


In [2]:
number_sample = 300
save_fig=False
plot_fig=True 

In [3]:
# flightPathLength = 110 # mm 
# # The initial value for t_0
# t0 = 51.74 # ns

# dataset name
# AL_data_b
# OLO_AL_6_data
# OLO_W_6_data
# OLO_Ni_8_data
# X6Cr17_2V30Min_5_data
# data_1029_Jun-27-2022_15-53_Pd
# data_115_Jul-27-2022_17-44_Powersweep3

In [4]:
tdc, pulse_mode, dataset, flightPathLength_d, t0_d = wd.dataset_tdc_selection()
display(tdc, pulse_mode, dataset, flightPathLength_d, t0_d)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

Text(value='OLO_AL_6_data', description='Dataset:', placeholder='Paste ticket description here!')

Text(value='110', description='Flight path length:', placeholder='Flight path length')

Text(value='51.74', description='t0:', placeholder='T_0 of the instrument')

$$\textbf{You can specify which dataset to use in below block}$$

In [5]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value

dataset_name = dataset.value

flightPathLength = float(flightPathLength_d.value) # mm 
# The initial value for t_0
t0 = float(t0_d.value) # ns

p = path.abspath(path.join("", "../../../.."))

variables.init()

path_main = os.path.join(p, 'tests//data')
filename_main = path_main + '//' + dataset_name + '.h5'

variables.path = os.path.join(p, 'tests//results//load_crop')
variables.result_path = os.path.join(p, 'tests/results/ion_type_selection/' + dataset_name)
if not os.path.isdir(variables.result_path):
        os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = variables.path + '//' + dataset_name + '//' + dataset_name + '_cropped' + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)


In [6]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp
0,5239.389648,5239.389648,20476.0,626.265702,5.762449,4.329796,0.0,1.0
1,5239.389648,5239.389648,21274.0,648.087858,-11.206531,26.902041,798.0,1.0
2,5239.389648,5239.389648,22584.0,639.583938,24.259592,15.504490,1310.0,1.0
3,5239.389648,5239.389648,22746.0,627.959628,18.083265,-7.545306,162.0,1.0
4,5239.389648,5239.389648,23789.0,623.357910,-1.942041,-2.706122,1043.0,1.0
...,...,...,...,...,...,...,...,...
1927581,5710.397949,5710.397949,19724.0,609.401880,-15.886531,-9.742041,77.0,1.0
1927582,5710.397949,5710.397949,19742.0,601.796358,11.397551,-5.475918,18.0,1.0
1927583,5710.397949,5710.397949,19969.0,455.220324,31.008980,-1.719184,227.0,1.0
1927584,5710.397949,5710.397949,20649.0,611.857044,2.483265,-27.284082,680.0,1.0


In [20]:
# exctract needed data from Pandas data frame as an numpy array
dld_highVoltage = data['high_voltage (V)'].to_numpy()
if pulse_mode_ini == 'voltage':
    dld_pulseVoltage = data['pulse (V)'].to_numpy()
elif pulse_mode_ini == 'laser':
    dld_pulseVoltage = data['pulse (deg)'].to_numpy()
dld_t = data['t (ns)'].to_numpy()
dld_x = data['x (mm)'].to_numpy()
dld_y = data['y (mm)'].to_numpy()
#mc = np.array(hdf["cropped_dld/mc"])

In [21]:
def mc_his(bin_size, prominence, distance, percent, plot, figname, mc, mc_ideal):

    bin_size = float(bin_size)
    prominence = int(prominence)
    distance = int(distance)
    percent = int(percent)
    if mc_ideal == True:
        mc_ideal = np.array(variables.listMaterial)
        print('List of mc are:', mc_ideal)
    else:
        mc_ideal = np.zeros(0)
    max_hist, left_right_peaks, peaks_sides, max_paek_edges, index_max = tools.massSpecPlot(mc[mc < 100], bin_size, mc_ideal, distance=distance, percent=percent, prominence=prominence, plot=plot, label='tof', fig_name=figname)
    mrp = (max_hist / (left_right_peaks[1] - left_right_peaks[0]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_sides)):
        print('Peaks ', i, 'is: {:.2f}'.format(peaks_sides[i,0]), 'peak window sides are: {:.2f} - {:.2f}'.format(peaks_sides[i,2], peaks_sides[i,3]))
    return peaks_sides

In [22]:
peaks_sides = interact_manual(mc_his, bin_size=widgets.Textarea(value="0.1")
               ,prominence=widgets.Textarea(value="60"), distance=widgets.Textarea(value="100")
               ,percent=widgets.Textarea(value="20"), plot=fixed(True), mc=fixed(mc),  figname=fixed(figname)
                , mc_ideal=[('False',False),('True',True)]);



interactive(children=(Textarea(value='0.1', description='bin_size'), Textarea(value='60', description='promine…

In [7]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList,elementIsotopeList,elementMassList,abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0],element[1],element[3]),element[2],)
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList,"Elements")
dropdown.observe(wd.on_change)


chargeDropdown = wd.dropdownWidget(chargeList,"Charge")
chargeDropdown.observe(wd.on_change_charge)

wd.compute_element_isotope_values_according_to_selected_charge()

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")

display(dropdown)
display(chargeDropdown)
display(buttonAdd)
display(buttonDelete)
display(buttonReset)
listMaterial = buttonAdd.on_click(wd.onClickAdd)
buttonDelete.on_click(wd.onClickDelete)
buttonReset.on_click(wd.onClickReset)

Dropdown(description='Elements', options=(('H (1) (99.98)', 1.01), ('H (2) (0.01)', 2.01), ('He (3) (0.00)', 3…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

In [ ]:
# add label to to hdf file
data_loadcrop.add_tof_mc_data_to_hdf5(filename_main, dld_t_calib_final, mc_p_1)
data_loadcrop.add_tof_mc_data_to_hdf5(filename, dld_t_calib_final, mc_p_1)
